In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import numba as nb
import cython as cy
import scipy
import const 
from scipy.special import gamma
from scipy.special import kv
from scipy import *
from scipy.integrate import odeint
import math as m
import mpmath

In [ ]:
def min(a, b):
    if a < b:
        return a
    else:
        return b

#TODO: Find temp true value
temp = 1e6
    
zeta = const.m_p * const.c**2 / (const.k_B * temp)

In [ ]:
#Définition des fonctions de Bessel de première espèce (avec une limite finie en 0)

K1p = lambda x: mpmath.besselk(1,x)
K2p = lambda x: mpmath.besselk(2,x)
K3p = lambda x: mpmath.besselk(3,x)

K1 = np.vectorize(K1p)
K2 = np.vectorize(K2p)
K3 = np.vectorize(K3p)

'''
def K1(x) :
    return kv(1, x)

def K2(x) :
    return kv(2, x)

def K3(x) :
    return kv(3, x)

'''

def gamma_maj(ze):
    return float((K3(ze)/K2(ze) - 1/ze))

def gamma_chapeau_prime(ze):
    a = ze*(3*K3(ze) + K1(ze) - 4*K2(ze))
    return float(1 + 4*K2(ze)/a)





In [ ]:
#Def des constantes 

tetav=20*3.14/180
c=3*10**8
mp = 1.67e-27
sh_max=100
E=10**43
n0=10**(3)  #let's say 1 particle by cubic cm



In [ ]:
def mu(teta, tetav, phi):
    return np.sin(teta) * np.sin(tetav) * np.cos(phi) + np.cos(teta) * np.cos(tetav)


def C_BM(E, n0):
    return np.sqrt(17 * E / (8 * np.pi * n0 * const.m_p * const.celer**5))

def C_ST(E, n0):
    return (2/5) * 1.15 * (E / (n0 * const.m_p * const.celer**5))**(1/5)




#TODO: Petit ou grand t ici ? 

def radius(T, teta, tetav, phi, E, n0, sh_max):
    C_BM_val = C_BM(E, n0)
    C_ST_val = C_ST(E, n0)
    mu_val = mu(teta, tetav, phi)
    def f(R, t):
        t =T+ mu_val * R / const.celer
        mine = np.minimum((C_BM_val**2) * (t) ** (-3) + (C_ST_val**2) * (t) ** (-6 / 5), sh_max**2)
        return np.sqrt(mine / (1 + mine)) * const.celer
    
    Rp = -T*const.celer/mu_val
    Rs = 1e25
    cpt = 0
    while abs(Rp - Rs)/abs(Rs) > 10 ** (-2) and cpt < 20:
        cpt += 1
        #print(Rs)
        time1 = [0,T+mu_val*Rs/const.celer]
        sol = odeint(f,y0=[-T*const.celer/mu_val], t=time1)
        Rp = Rs
        Rs = sol[1][0]
    return Rs

def shock_speed(E, n0, sh_max, T, teta, tetav, phi):
    mu_val =mu(teta, tetav, phi)
    radius_value = radius(T, teta, tetav, phi, E, n0, sh_max)
    t = T + mu_val * radius_value / const.celer
    mine = min((C_BM(E, n0)**2) * (t) ** (-3) + (C_ST(E, n0)**2) * (t) ** (-6/5), sh_max**2)
    beta_sh = np.sqrt(mine / (1 + mine))
    return beta_sh

#une fois R déterminé on a t=T+mu*R/c l'équation 1 nous donne alors beta sh donc la vitesse de l'onde de choc




#pour un T fixé sur Terre, l'observateur reçoit des photons de différents points de passage de l'onde de choc. Les points sont repérés en coordonnées polaires
#par teta phi et teta v, il s'agit de savoir en quel t l'onde de choc passe en un point de coordonnées (tetav,teta,phi) et émet un photon reçu en T sur terre
# Définition de la grille en theta et phi
phi_values = np.linspace(0, 2 * np.pi, 10)
theta_values = np.linspace(0, 3*np.pi/180, 10)  #angle d ouverture du jet borné à 3 deg
phi_grid, theta_grid = np.meshgrid(phi_values, theta_values)



#TODO: reprendre en fonction des noueaux paramètres de raduis et shock_speed
# Choix de la valeur de T
T = 8640000
# Calcul des valeurs de t pour chaque point de la grille
t_values = np.zeros_like(phi_grid)
r_values = np.zeros_like(phi_grid)
nb_pt_traite = 0
for i in range(len(phi_values)):
    for j in range(len(theta_values)):
        nb_pt_traite += 1
        mu = np.sin(theta_values[j]) * np.sin(tetav) * np.cos(phi_values[i]) + np.cos(theta_values[j]) * np.cos(tetav)
        R_values = radius(T, mu, c)
        r_values[j, i] = R_values 
        
        t_values[j, i] = T + mu * R_values / c
        if nb_pt_traite%1000==0:
            print(nb_pt_traite/100)
            print(mu)
t_values = t_values/86400
# Tracé de la grille en coordonnées sphériques (theta, phi) avec les valeurs de t en code couleur
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='polar')
plt.title(f"EATS pour T={T/(3600*24)} jours")
im=ax.contourf(phi_grid,theta_grid,t_values)
fig.colorbar(im)

#Now we can calculate the value of gamma_maj :








In [ ]:
# Détermination de Ksi, car il est défini implicitement ici. 
#Find the rpoblem with ksi ici
from scipy.optimize import fsolve

def eq(gamma_sh,ze):
    a = (gamma_maj(ze) + 1)*(1 + gamma_chapeau_prime(ze)*(gamma_maj(ze) - 1))**2
    b = 2 + gamma_chapeau_prime(ze)*(2 - gamma_chapeau_prime(ze)*(gamma_maj(ze) - 1))
    return gamma_sh**2 - a/b

def sol(gamma_sh): #Sol is zeta
    def func(ze):
        return eq(gamma_sh, ze)
    solve = fsolve(func, 1)
    return solve[0]

sol_vect = np.vectorize(sol)
g_m = np.vectorize(gamma_maj)
g_c = np.vectorize(gamma_chapeau_prime)


#We can now calculate values of beta's

def beta_u(E, n0, sh_max, T):
    return shock_speed(E, n0, sh_max, T)

def beta(E, n0, sh_max,T):
    beta_sh = shock_speed(E, n0, sh_max,T)
    zeta = sol(beta_sh)
    return np.sqrt(1 - 1/(gamma_maj(zeta) ** 2))

def beta_d(E, n0, sh_max,T):
    beta_sh = shock_speed(E, n0, sh_max,T)
    return (beta_sh - beta(E, n0, sh_max,T)) / (1 - beta(E, n0, sh_max,T) * beta_sh)

def p(E, n0, sh_max, T):
    beta_u_val = beta_u(E, n0, sh_max, T)
    beta_d_val = beta_d(E, n0, sh_max, T)
    a = 3*beta_u_val - 2*beta_u_val*(beta_d_val**2) + beta_d_val**3
    b = beta_u_val - beta_d_val
    return a/b - 2

def gamma_shf(E, n0, sh_max,T):
    beta_sh = shock_speed(E, n0, sh_max,T)
    return np.sqrt(1/(1 - beta_sh**2))

def n_prime(gamma_sh, n0):
    return 4*gamma_maj(sol(gamma_sh))*n0

def e_i_p(n0, gamma_sh):
    return const.m_p*((const.celer)**2)(gamma_maj(sol(gamma_sh)) - 1)*n_prime(n0,gamma_sh)

epsi_b = 4.12*(1/np.power(10,5))
def gamma_prime_c(n0, epsilon_B, gamma_sh,t):
    return (3 * const.m_e * const.celer* gamma_maj(sol(gamma_sh))) / (4 * const.sigma_T * epsilon_B *e_i_p(gamma_sh) * t*n0)


def gamma_prime_m(E, n0, sh_max, T, gamma_sh):
    p_val = p(E, n0, sh_max, T)
    arg = ((p_val - 2)*const.m_p/(p_val - 1)*const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1)
    return np.max(1, arg)

def B_prime(epsilonB, gamma_sh, n0):
    return (np.sqrt(8*const.pi*epsilonB*e_i_p(gamma_maj,gamma_chapeau_prime, gamma_sh,n0)))

def nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T):
    p_val = p(E, n0, sh_max, T)
    return 3/16 * gamma_prime_m(p_val,gamma_sh)**2 * const.q_e*B_prime(epsilon_B, gamma_sh,n0) / (const.m_e * const.celer)

def nu_prime_c(n0,epsilon_B, t, gamma_sh):
    return 3/16 * gamma_prime_c(n0,epsilon_B,gamma_sh, t)**2 * const.q_e *B_prime(epsilon_B, gamma_sh, n0) / (const.m_e * const.celer)


In [ ]:
def power_I(x,y):
    return 1/np.power(x,y)

#Rapport de comparaison pour les refroidissemts entre nu_prim_c et nu_prim_m
def Kappa(n0,epsilon_B,t,gamma_sh):
    return nu_prime_c(n0, epsilon_B, gamma_sh,t)/nu_prime_m(epsilon_B, gamma_sh, n0)

def n_prime_r(gamma_sh, E, n0, sh_max, T):
    beta_u_val = beta_u(E, n0, sh_max, T)
    beta_d_val = beta_d(E, n0, sh_max, T)
    p_val = p(E, n0, sh_max, T)
    return n_prime(gamma_sh,n0)*min(1,((p_val - 2)*const.m_p/(p_va - 1)
                                       *const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1))

def FastcEmission(n0,x,p,epsilon_B,t,gamma_sh):
    if x > Kappa(n0,epsilon_B,t,gamma_sh):
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh),-1/3)*np.power(x,1/3)
    elif x >= Kappa(n0,epsilon_B,t,gamma_sh) and x <1:
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh),1/2)*power_I(x,-1/2)
    else:
        power_I(Kappa(n0,epsilon_B,t,gamma_sh),-1/2)*power_I(x,-p/2)

#Cas du slow-cooling
def SlowcEmission(x,p,epsilon_B,t,gamma_sh,n0):
    if x < 1:
        return np.power(x,1/3)
    elif x>=1 and x < Kappa(n0,epsilon_B,t,gamma_sh):
        return power_I(x,-(p-1)/2)
    else:
        return power_I(Kappa(n0,epsilon_B,t,gamma_sh),-1/2)*power_I(-p/2)




#Fonction d'émission
def Emission_function(n0,x,epsilon_B,t,gamma_sh,p):
    if Kappa(n0,epsilon_B,t,gamma_sh) > 1:
        return SlowcEmission(x,Kappa,p,epsilon_B,t,gamma_sh,n0)
    else:
        return FastcEmission(n0,x, Kappa,p,epsilon_B,t,gamma_sh)


def fact_denorm(beta_u,beta_d,n0,gamma_sh,epsilon_b):
    return 0.88*(256*(p(beta_u,beta_d) - 1)*(const.q_e**3)*n_prime_r(n0,gamma_sh,beta_u,beta_d)
                 *B_prime(epsilon_b,gamma_sh,n0)/27*(3*(p(beta_u,beta_d) - 1)*const.m_e*(const.celer**2)))

In [ ]:
#To_DO ABSOLUMENT reprendre, il existe des fonctions nu_prime_r etc...

#gamma_prime_m_val = gamma_prime_m(2.05, gamma_sh)
#gamma_prime_c_val = gamma_prime_c(n0, epsi_b_gaussianJ, 10**5, gamma_sh)
#n_prime_r_val = n_prime_r(n0,gamma_sh, beta_u,beta_d)
#b_prime = B_prime(epsi_b_gaussianJ, gamma_sh,n0)
#nu_prime_c_val = nu_prime_c(n0, epsi_b_gaussianJ, 10**5, gamma_sh)
#nu_prime_m_val = nu_prime_m(epsi_b_gaussianJ, gamma_sh,n0,2.05)
#TODO: verify expression
def p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T):
    p_val = p(beta_u(E, n0, sh_max, T), beta_d(E, n0, sh_max, T))
    if gamma_prime_m(p, gamma_sh) >= gamma_prime_c(n0, epsilon_B, gamma_sh, t):
        return p_val 
    else:
        beta_u_val = beta_u(E, n0, sh_max, T)
        beta_d_val = beta_d(E, n0, sh_max, T)
        return (3 * beta_u_val - 2 * beta_u_val * beta_d_val ** 2 + beta_d_val ** 3) / (beta_u_val - beta_d_val) - 2

# n_prime_r et b_prime a ne pas mettre en global.
#gamma est la fonction gamma d'euler implémenté grâce à scipy. 
def c_11(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T):
    nu_prime_c_val = nu_prime_c(n0, epsilon_B, t, gamma_sh)
    nu_prime_m_val = nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T)
    gamma_prime_m_val = gamma_prime_m(E, n0, sh_max, T, gamma_sh)
    gamma_prime_c_val = gamma_prime_c(n0, epsilon_B, gamma_sh, t)
    n_prime_r_val = n_prime_r(gamma_sh, E, n0, sh_max, T)
    b_prime = B_prime(epsilon_B, gamma_sh,n0)
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T)
    
    nu_min = min(nu_prime_m_val, nu_prime_c_val)
    gamma_min = min(gamma_prime_m_val, gamma_prime_c_val)
    return 2 ** 6 *const.pi ** (5/6) * (p_tildt_val + 2) * (p_tildt_val - 1) * const.q_e * n_prime_r_val / (15 * (3*p_tildt_val + 2) * gamma(5 / 6) * gamma_min ** 5 * b_prime) * (nu_prime / nu_min) ** (-5/3)

def c_14(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T):
    nu_prime_c_val = nu_prime_c(n0, epsilon_B, t, gamma_sh)
    nu_prime_m_val = nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T)
    gamma_prime_m_val = gamma_prime_m(E, n0, sh_max, T, gamma_sh)
    gamma_prime_c_val = gamma_prime_c(n0, epsilon_B, gamma_sh, t)
    n_prime_r_val = n_prime_r(gamma_sh, E, n0, sh_max, T)
    b_prime = B_prime(epsilon_B, gamma_sh, n0)
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max, T)
    nu_min = min(nu_prime_m_val, nu_prime_c_val)
    gamma_min = min(gamma_prime_m_val, gamma_prime_c_val)
    return (2 ** ((3*p_tildt_val + 8) / 2) * (p_tildt_val - 1) * gamma(3/2 + p_tildt_val/4) * gamma(1/6 + p_tildt_val/4) * const.q_e * n_prime_r_val ) / (3**(3/2) * const.pi **((p_tildt_val + 1 )/2) * gamma(2 + p_tildt_val/4) * gamma_min ** 5 *b_prime) *(nu_prime / nu_min) ** (-(p_tildt_val + 4) / 2)

def nu_prime_zero(gamma_sh, n0, epsilon_B, t, E, sh_max,T):
    nu_prime_c_val = nu_prime_c(n0, epsilon_B, t, gamma_sh)
    nu_prime_m_val = nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T)
    nu_prime_min = min(nu_prime_m_val, nu_prime_c_val)
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T)
    return ((5*(3 * p_tildt_val +2) * gamma(3/2 + p_tildt_val/4) *gamma(11/6 + p_tildt_val/4) * gamma(1/6 + p_tildt_val/4) * gamma(5/6)) / (p_tildt_val + 2) * gamma(2 + p_tildt_val/4))** (6 / (3*p_tildt_val+2)) * ((2**( 3 * (3 * p_tildt_val -4))) / (27 * const.pi ** (3*p_tildt_val + 8) )) ** (1 / (3*p_tildt_val + 2)) * nu_prime_min

def alpha_prime_nu_prime(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T):
    nu_prime_m_val = nu_prime_m(epsilon_B, gamma_sh, E, n0, sh_max, T)
    nu_prime_c_val = nu_prime_c(n0, epsilon_B, t, gamma_sh)
    p_tildt_val = p_tildt(gamma_sh, n0, epsilon_B, t, E, sh_max,T)
    nu_min = min(nu_prime_m_val, nu_prime_c_val)
    nu_zero = nu_prime_zero(gamma_sh, n0, epsilon_B, t, E, sh_max,T)
    if nu_prime < nu_zero:
        alpha_prime_nu_prime_zero = c_11(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T)
        eta =  -5/3
    else : 
        alpha_prime_nu_prime_zero = c_14(nu_prime, gamma_sh, n0, epsilon_B, t, E, sh_max,T)
        eta = -(p_tildt_val + 4) / 2
    return alpha_prime_nu_prime_zero * (nu_prime / nu_zero) ** eta

def nu_prime(nu, xi, beta, mu):
    return gamma_maj(xi) * (1 - beta * mu) * nu 




In [ ]:


def alpha_nu(mu, E, n0, sh_max,T, nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c, p_tildt): #zeta imported or not ? 
    beta_sh = shock_speed(E, n0, sh_max, T)
    beta = beta(E, n0, sh_max, T)
    alpha_prime_nu_prime_val = alpha_prime_nu_prime(nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c, p_tildt)
    zeta = sol(beta_sh)
    alpha_nu = gamma_maj(zeta)*(1-beta*mu)*alpha_prime_nu_prime_val
    return alpha_nu

def delta_s(R, mu, E, n0, sh_max,T):
    beta_sh = shock_speed(E, n0, sh_max,T)
    zeta = sol(beta_sh)
    return R/((12*np.abs(mu - beta_sh)*gamma_maj(zeta))**2)

def tan_nu(R, mu, E, n0, sh_max,T, nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c, p_tildt):
    alpha_nu_val = alpha_nu(E, n0, sh_max,T, mu, nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c, p_tildt)
    delta_s_val = delta_s(R, mu, E, n0, sh_max, T)
    return alpha_nu_val * delta_s_val

In [ ]:
#Calcul de l'Intégrande pour un T donné:
def Integrand(nu,n0, epsi_b, espi_e, teta_v, D, T, teta,phi, nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c, p_tildt, E, R, sh_max, ):
    t = T + mu(teta, tetav, phi)*radius(T,teta,teta_v,phi)/const.celer # Pour un certain angle polaire, pour un certain temps de l'observateur, on détermine t
    gamma_sh_val = gamma_shf(E,n0,sh_max,T)
    zeta_val = sol(gamma_sh_val)
    p_val = p(E, n0, sh_max, T)
    nu_p_val = nu*gamma_maj(zeta_val)*(1-beta(E,n0,sh_max,T)*mu(teta,teta_v,phi))
    nu_prime_m_val = nu_prime_m(epsi_b, gamma_sh_val,n0,p_val)
    x_val = nu_p_val/nu_prime_m(epsi_b, gamma_sh_val,n0,p_val)
    energ_normalized = Emission_function(n0,x_val,epsi_b,t,gamma_sh_val,p_val)
    energ = energ_normalized*fact_denorm(beta_u(E, n0,sh_max,T), beta_d(E, n0,sh_max,T),n0,gamma_sh_val,epsi_b) #Calcul de l'énergie pour la fréquence nu'
    alpha_prime_nu_prime_val = alpha_prime_nu_prime(nu_p_val, nu_prime_c, nu_prime_m_val, gamma_prime_m, gamma_prime_c, p_tildt)
    alpha_nu_val = alpha_nu(mu, E, n0, sh_max, T, nu_p_val, nu_prime_c, nu_prime_m_val, gamma_prime_m, gamma_prime_c, p_tildt)
    delta_s_val = delta_s(R, mu(teta, tetav, phi), E, n0, sh_max, T)
    tau_nu = alpha_nu_val * delta_s_val #Pour la suite on calcul la partie liée aux agnles et à la vitesse du choc
    angular_sh = np.abs(mu(teta, tetav, phi) - shock_speed(E,n0,sh_max,T,teta, tetav, phi))*np.power(radius(T,teta,tetav,phi,E,n0,sh_max),2)/(1-mu(teta, tetav, phi)*shock_speed(E,n0,sh_max,T,teta, tetav, phi))*np.power(gamma_maj(zeta_val)*(1 - beta(E,n0,sh_max,T)*mu(teta, tetav,phi)),3)
    
    